In [12]:
import os

os.environ["EB_AGENT_LOGGING_LEVEL"] = "INFO"
os.environ["EB_AGENT_ACCESS_TOKEN"] = "4ce50e3378f418d271c480c8ddfa818537071dbe"

from pydantic import Field
from typing import Dict, Type, List, Any

from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, Message, AIMessage, FunctionMessage, SlidingWindowMemory
from erniebot_agent.tools.base import Tool
from erniebot_agent.tools.schema import ToolParameterView
from erniebot_agent.agents import FunctionAgent
from erniebot_agent.tools import RemoteToolkit
from erniebot_agent.file import GlobalFileManagerHandler



In [15]:
import os
from erniebot_agent.agents.function_agent_with_retrieval import FunctionAgentWithRetrieval
from erniebot_agent.memory.whole_memory import WholeMemory
from erniebot_agent.chat_models.erniebot import ERNIEBot
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredMarkdownLoader
from erniebot_agent.extensions.langchain.embeddings import ErnieEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from erniebot_agent.tools import RemoteToolkit

embeddings = ErnieEmbeddings(aistudio_access_token=os.environ["EB_AGENT_ACCESS_TOKEN"] , chunk_size=16)

class FaissSearch:
    def __init__(self, db, embeddings):
        # 类的初始化方法，接收一个数据库实例并将其存储在类的实例变量 self.db 中，接收一个embeddings方法传到self.embeddings中
        self.db = db
        self.embeddings = embeddings

    def search(self, query: str, top_k: int = 10, **kwargs):
        # 定义一个搜索方法，接受一个查询字符串 'query' 和一个整数 'top_k'，默认为 10
        docs = self.db.similarity_search(query, top_k)
        # 调用数据库的 similarity_search 方法来获取与查询最相关的文档
        para_result = self.embeddings.embed_documents([i.page_content for i in docs])
        # 对获取的文档内容进行嵌入（embedding），以便进行相似性比较
        query_result = self.embeddings.embed_query(query)
        # 对查询字符串也进行嵌入
        similarities = cosine_similarity([query_result], para_result).reshape((-1,))
        # 计算查询嵌入和文档嵌入之间的余弦相似度
        retrieval_results = []
        for index, doc in enumerate(docs):
            retrieval_results.append(
                {"content": doc.page_content, "score": similarities[index], "title": doc.metadata["source"]}
            )
        # 遍历每个文档，将内容、相似度得分和来源标题作为字典添加到结果列表中
        return retrieval_results  # 返回包含搜索结果的列表

In [17]:
md_file_path = '/Users/tanzhehao/Documents/ERINE/ERNIE-Bot-SDK/docs/modules/file.md'
documents = UnstructuredMarkdownLoader(file_path=md_file_path)
text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size=320, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)


TypeError: 'UnstructuredMarkdownLoader' object is not iterable

In [9]:
def read_md_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            md_content = file.read()
        return md_content
    except FileNotFoundError:
        print(f"文件 '{file_path}' 不存在。")
        return None
    except Exception as e:
        print(f"读取文件时出现错误： {e}")
        return None

# 例子
md_file_path = '/Users/tanzhehao/Documents/ERINE/ERNIE-Bot-SDK/docs/modules/file.md'
content = read_md_file(md_file_path)

if content:
    print(content)


# File 模块

## 1. 模块简介

文件管理模块提供了用于管理文件的一系列类，方便用户与Agent进行交互，其中包括 `File` 基类及其子类、`FileManager` 、`GlobalFileManagerHandler`以及与远程文件服务器交互的  `RemoteFileClient`。

推荐使用  `GlobalFileManagerHandler`在事件循环开始时初始化 `FileManager`以及获取全局的 `FileManager`，之后只需通过这个全局的 `FileManager`对文件进行增、删、查等操作以及获取Agent产生的文件。

!!! notes 注意

    - **不推荐**用户自行操作 `File`类以免造成资源泄露。

    - `FileManager`操作文件主要用于异步函数中，在同步函数中使用可能会无效。

    - `FileManager`将作为此模块中生命周期最长的对象，它会在关闭时回收所有的持有对象（RemoteClient/temp local file），请不要随意关闭它。如果需要关闭，需要先停止对其中注册文件的使用。

## 2. 核心类

下面简单介绍 `File` 模块的核心类，详细接口请参考[API文档](../package/erniebot_agent/file.md)。

### 2.1 File 基类

`File` 类是文件管理模块的基础类，用于表示通用的文件对象（不建议自行创建 `File` 类以免无法被 `Agent`识别使用以及无法被回收）。它包含文件的基本属性，如文件ID、文件名、文件大小、创建时间、文件用途和文件元数据。

此外， `File`类还定义了一系列抽象方法，比较常用的有：

* 异步读取文件内容的 `read_contents`方法
* 将文件内容写入本地路径的 `write_contents_to`方法

以及一些辅助方法：

* 生成文件的字符串表示形式
* 转换为字典形式

在File类的内部，其主要有两个继承子类，一个是 `Local File`，一个是 `Remote File`。

以下是 `File` 基类的属性以及方法介绍：

| 属性       | 类型           | 描述                         

In [10]:
content

'# File 模块\n\n## 1. 模块简介\n\n文件管理模块提供了用于管理文件的一系列类，方便用户与Agent进行交互，其中包括 `File` 基类及其子类、`FileManager` 、`GlobalFileManagerHandler`以及与远程文件服务器交互的  `RemoteFileClient`。\n\n推荐使用  `GlobalFileManagerHandler`在事件循环开始时初始化 `FileManager`以及获取全局的 `FileManager`，之后只需通过这个全局的 `FileManager`对文件进行增、删、查等操作以及获取Agent产生的文件。\n\n!!! notes 注意\n\n    - **不推荐**用户自行操作 `File`类以免造成资源泄露。\n\n    - `FileManager`操作文件主要用于异步函数中，在同步函数中使用可能会无效。\n\n    - `FileManager`将作为此模块中生命周期最长的对象，它会在关闭时回收所有的持有对象（RemoteClient/temp local file），请不要随意关闭它。如果需要关闭，需要先停止对其中注册文件的使用。\n\n## 2. 核心类\n\n下面简单介绍 `File` 模块的核心类，详细接口请参考[API文档](../package/erniebot_agent/file.md)。\n\n### 2.1 File 基类\n\n`File` 类是文件管理模块的基础类，用于表示通用的文件对象（不建议自行创建 `File` 类以免无法被 `Agent`识别使用以及无法被回收）。它包含文件的基本属性，如文件ID、文件名、文件大小、创建时间、文件用途和文件元数据。\n\n此外， `File`类还定义了一系列抽象方法，比较常用的有：\n\n* 异步读取文件内容的 `read_contents`方法\n* 将文件内容写入本地路径的 `write_contents_to`方法\n\n以及一些辅助方法：\n\n* 生成文件的字符串表示形式\n* 转换为字典形式\n\n在File类的内部，其主要有两个继承子类，一个是 `Local File`，一个是 `Remote File`。\n\n以下是 `File` 基类的属性以及方法介绍：\n\n| 属性       | 类型 

In [8]:
from erniebot_agent.retrieval import BaizhongSearch
knowledge_base = BaizhongSearch(access_token='4ce50e3378f418d271c480c8ddfa818537071dbe', knowledge_base_name='my_db')
# knowledge_base.search(step_input, top_k=self.top_k, filters=None)
# knowledge_base.create_knowledge_base(knowledge_base_name='my_db')

BaizhongError: error message is request error: {"code":"MissingDateHeader","message":"Request must have a \"date\" or \"x-bce-date\" header.","requestId":"ac2563e6-d417-4aa3-8a9e-fc523a5dabc6"} error code is 404

In [6]:
knowledge_base

NameError: name 'knowledge_base' is not defined